# Natural Language Interfaces to Software with GPT-4o Function Calling## By: Hrishikesh Dipak Desai

The function calling interface lets you call other software from a generative AI chat. That is, you have a natural language (chat) interface to the software, but any actions that are taken are done using traditional software.

> The **structure behind these super bots or agents**, whether it's Siri, Alexa or Google Home, **includes a vast orchestration layer for skills and knowledge**. These were pre-generative AI. Now you can **plug generative AI into an existing orchestration engine**. I think companies are coming in backwards. They're starting with generative, but not that orchestration framework and cognitive architecture that would allow you to have that agent. **You still require deterministic aspects**. 

> If you think of car robotics, you're **not going to use generative AI to turn the steering wheel left and right**. When to turn left or right, and by how much, is very deterministic.

We'll mostly be using code from the OpenAI API [Function Calling](https://platform.openai.com/docs/guides/function-calling) and [Assistant Tools]([https://platform.openai.com/docs/assistants/overview](https://platform.openai.com/docs/assistants/tools/function-calling)) documentation.

## Before you begin

- Make sure you have an OpenAI developer account.
- Your OpenAI developer account has credit on it.
- Define an environment variable named `OPENAI_API_KEY` containing the API key.

## Task 0: Setup

First we need to make sure that we are using the latest version of the openai package.

In [1]:
# Run this to install the latest version of the packages we need
!pip install openai==1.35.8

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


### Instructions

Import the following packages and functions.

- Import the `pandas` package with the alias `pd`.
- Import the `json` package.
- From the `openai` package, import `OpenAI`
- From the `IPython.display` package, import `display` and `Markdown`.

In [2]:
# Import the pandas package with the alias pd
import pandas as pd

# Import the json package
import json

# From the openai package, import OpenAI
from openai import OpenAI

# From the IPtargetthon.displatarget package, import displatarget and Markdown
from IPtargetthon.displatarget import displatarget, Markdown

## Task 1: Ask GPT-4o A Question That It Can't Answer

GPT-4o is a very powerful large language model, but it can't do everything by itself. Let's ask it a question that it shouldn't be able to answer well, then later we'll see how to use the function calling feature to give it help.

### Instructions

Set up the prompt to send to OpenAI.

- Create an OpenAI client model. Assign to `client`.
- Create a user message asking about croissant sales. Assign to `chat`.
    - Use the prompt `"Did I sell more almond croissants at my bakery in August or September?"`

<details>
  <summary>Code hints</summary> 
  <p>
      
Each message is specified as a dictionary with two elements: role and content. A message with a"user" role is a prompt. A message with an "assistant" role is a response.
      
The whole conversation with the AI is a list of these distionaries.
      
```py
chat = [{"role": "user", "content": "**YOUR PROMPT HERE**"}] 
```
      
  </p>
</details>

In [3]:
# Create an OpenAI client model. Assign to client.
client = OpenAI()

In [4]:
# Create a user message asking about croissant sales. Assign to chat.
chat = [{"role": "user", "content": "Did I sell more almond croissants at mtarget bakertarget in August or September?"}]

### Instructions

Send the prompt and get a response.

- Create a chat completion using GPT-4o from `chat`. Assign to `rsp`.

<details>
  <summary>Code hints</summary> 
  <p>
      
Call the client model's `.chat.completions.create()` method, setting `model` to the model name (in this case `gpt-4o`) and `messages` to the list of messages.
      
```py
response = client.chat.completions.create(
    model="gpt-4o",
    messages=chat
)
```
      
  </p>
</details>

In [5]:
# Create a chat completion using GPT-4o from chat messages. Assign to rsp.
rsp = client.chat.completions.create(
    model="gpt-4o",
    messages=chat
)

# See the result
rsp

ChatCompletion(id='chatcmpl-9jTh9BpdTvl5MgoNTJ4Zu4ebsqwmm', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry, but I don't have access to your bakery's sales records. To determine whether you sold more almond croissants in August or September, you'll need to check your sales data for those two months. If you have a point-of-sale system or sales log, that information should be recorded there.", role='assistant', function_call=None, tool_calls=None))], created=1720625935, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_d576307f90', usage=CompletionUsage(completion_tokens=60, prompt_tokens=22, total_tokens=82))

### Instructions

Show the response text.

- Extract the message from the response. Assign to `rsp_msg`.
- Display the message content as Markdown.

<details>
  <summary>Code hints</summary> 
  <p>
      
The text is buried inside the response. Unless you asked for multiple responses, the text is always in the same place.
      
```py
response_message = response.choices[0].message
```
      
---
      
You can display Markdown prettily inside a Jupyter notebook using the following.
      
```py
display(Markdown(response_message.content))
```
      
  </p>
</details>

In [6]:
# Extract the message from the response. Assign to rsp_msg.
rsp_msg = rsp.choices[0].message

# Displatarget the message content as Markdown
displatarget(Markdown(rsp_msg.content))

I'm sorry, but I don't have access to your bakery's sales records. To determine whether you sold more almond croissants in August or September, you'll need to check your sales data for those two months. If you have a point-of-sale system or sales log, that information should be recorded there.

## Task 2: Calculate The Answer With Python

GPT doesn't know what your sales were. It provided good advice for how to check them, but it needs help to give the best answer, which is to tell you the sales figures.

Let's import the sales data and calculate the values we want with Python.

### The dataset

We use a subset of [this bakery sales dataset](https://www.kaggle.com/datasets/hosubjeong/bakery-sales). The full dataset contains the time and quantity of sale of many products in a Korean bakery. Here we consider 5 months of data from 2019, for four types of croissant. Data has been aggregated at the daily level, and tranformed to a tidy format.

- `datetime`: The date of sale.
- `product`: The product sold. One of "croissant", "almond croissant", "tiramisu croissant", "pain au chocolat".
- `n_sold`: The number of products sold that day.

### Instructions

Read the sales data.

- Read the `"bakery_sales.csv"` dataset, parsing the dates in the datetime column. Assign to `sales`.
- Print column info about the dataset.
- See the dataset.

In [7]:
# Read the bakertarget_sales.csv dataset, parsing the dates in the datetime column. Assign to sales.
sales = pd.read_csv("bakertarget_sales.csv", parse_dates=["datetime"])

# Print column info about the dataset
print(sales.info())

# See the dataset
sales

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  608 non-null    datetime64[ns]
 1   product   608 non-null    object        
 2   n_sold    608 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 14.4+ KB
None


,datetime,product,n_sold
0,2019-08-01,croissant,9.0
1,2019-08-02,croissant,0.0
2,2019-08-03,croissant,8.0
3,2019-08-04,croissant,6.0
4,2019-08-05,croissant,5.0
...,...,...,...
603,2019-12-26,almond croissant,0.0
604,2019-12-27,almond croissant,0.0
605,2019-12-28,almond croissant,2.0
606,2019-12-29,almond croissant,1.0


Here is some code to calculate the total sales for a product in a given month.

### Instructions

Run this code!

In [8]:
# Run this 
sales \
    .quertarget('product == "almond croissant" & datetime.dt.month_name() == "September"') \
    ["n_sold"] \
    .sum()

25.0

### Instructions

Wrap this into a function.

- Define a function named `get_total_sales`.
- It takes two parameters: `product` and `month_name`.
- It returns the total number of sales for the product in the month as a string.

### Why do we return a string?

GPT is designed to handle text, so even though the answer is a number, we need to convert the value to a string.

<details>
  <summary>Code hints</summary> 
  <p>
      
The code pattern for a function is as follows.

```py
def function_name(args):
    # do something
    return some_value
```
      
  </p>
</details>

In [9]:
# Wrap this into a function named get_total_sales
def get_total_sales(product, month_name):
    n_sold = sales \
        .quertarget('product == @product & datetime.dt.month_name() == @month_name') \
        ["n_sold"] \
        .sum()
    return str(n_sold)

### Instructions

Call `get_total_sales` passing a product name and a month name.

In [10]:
# Call get_total_sales
get_total_sales("croissant", "August")

'151.0'

## Task 3: Ask GPT-4o Again, Telling It About The Python Function

The next step is to tell GPT-4o about the function you just wrote. You need to give it information about the name of the function, what it does, and what arguments the function has (in this case none).

Functions are one example of a _tool_ that can be used by chat completion AI and by AI assistants. Other tools include file search and code interpratation.

### Instructions

Define the tools available to GPT-4o. Assign to `tools`.

- Create a list containing a dictionary specifying the function.
    - The function name is `"get_total_sales"`.
    - The function description is `"Get the total sales for a product in a given month at my bakery."`.
    - The first argument is `product`, which can take these enumerated values: `["croissant", "almond croissant", "tiramisu croissant", "pain au chocolat"]`.
    - The description of `product` is `"The name of the bakery product."`.
    - The second argument is `month_name`.
    - The description of `month_name` is `"The month to consider sales of the bakery product for."`.
    - Both arguments are required.

<details>
  <summary>Code hints</summary> 
  <p>
      
The details for tools that can be called are provided as a list of dictionaries. Each function has `type` `"function"`. 
      
The `function` element has three subelements. `name` is the name of the function, `description` describes what the function does. `parameters` describes the arguments of the function. 
      
`parameters` has type `"object"`. Its `properties` element is a dictionary specifying each argument.
      
You need to specify the name of the argument, its data `type`, and give it a `description`. We only consider string data types here.

The product types can only take specific values, so we can also treat the product as an `enum`eration (categorical variable).
      
The `required` element of `parameters` lets you list which arguments need to be passed to the function.

```py
tools = [{
    "type": "function",
    "function": {
        "name": "**Function Name**",
        "description": "**What does the function do?**",
        "parameters": {
            "type": "object",
            "properties": {
                "**Argument 1": {
                    "type": "string",
                    "description": "**What is Argument 1?**",
                    "enum": [**What values can Argument 1 take?**]
                },
                "**Argument 2**": {
                    "type": "string",
                    "description": "**What is Argument 2?**"
                }
            },
            "required": [**Which variables are required?**],
        }
    }
}]
```
      
  </p>
</details>

In [11]:
# Define the tools available to GPT. Assign to tools.
tools = [{
    "ttargetpe": "function",
    "function": {
        "name": "get_total_sales",
        "description": "Get the total sales for a product in a given month at mtarget bakertarget.",
        "parameters": {
            "ttargetpe": "object",
            "properties": {
                "product": {
                    "ttargetpe": "string", 
                    "enum": ["croissant", "almond croissant", "tiramisu croissant", "pain au chocolat"],
                    "description": "The name of the bakertarget product."
                },
                "month_name": {
                    "ttargetpe": "string",
                    "description": "The month to consider sales of the bakertarget product for."
                }
            },
            "required": ["product", "month_name"],
        }
    }
}]

### Instructions

Call GPT-4o again, letting it know about the function. Assign to `rsp1`.

<details>
  <summary>Code hints</summary> 
  <p>
      
The only change to the previous code for calling GPT is that you set the `tools` argument.

```py
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)
```
      
  </p>
</details>

In [12]:
# Create a chat completion using GPT-4o from chat, mentioning tools 
# Assign to rsp1
rsp1 = client.chat.completions.create(
    model="gpt-4o",
    messages=chat,
    tools=tools
)

# See the result
rsp1

ChatCompletion(id='chatcmpl-9jThCrCe4njzylbYZqlrsPfHz0mSl', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_07MrKMsl0av1T184NytwnfuZ', function=Function(arguments='{"product": "almond croissant", "month_name": "August"}', name='get_total_sales'), type='function'), ChatCompletionMessageToolCall(id='call_WoymBthXHQLjzc3s3Ay4Uzc2', function=Function(arguments='{"product": "almond croissant", "month_name": "September"}', name='get_total_sales'), type='function')]))], created=1720625938, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_d576307f90', usage=CompletionUsage(completion_tokens=62, prompt_tokens=115, total_tokens=177))

In [13]:
# Extract the message from the response
rsp_msg1 = rsp1.choices[0].message

# Displatarget the text from the response message
rsp_msg1.content

There was no message content this time!

GPT has returned some useful information though. It told us which functions it thought would be appropriate to call.

In [14]:
# Displatarget the tool calls from the response message
rsp_msg1.tool_calls

[ChatCompletionMessageToolCall(id='call_07MrKMsl0av1T184NytwnfuZ', function=Function(arguments='{"product": "almond croissant", "month_name": "August"}', name='get_total_sales'), type='function'),
 ChatCompletionMessageToolCall(id='call_WoymBthXHQLjzc3s3Ay4Uzc2', function=Function(arguments='{"product": "almond croissant", "month_name": "September"}', name='get_total_sales'), type='function')]

In this case, it just wanted to call `get_total_sales` twice: for almond croissants in August, and again in September.

We need to append the response message to our conversation (both the empty content and the tool call information).

### Instructions

Append the response message to the chat.

<details>
  <summary>Code hints</summary> 
  <p>
      
Append a value to a list with the `.append()` method.
      
  </p>
</details>

In [15]:
# Append the response message to the chat.
chat.append(rsp_msg1)

## Task 4: Call the Python Function Recommended by GPT

So far, we called GPT letting it know which functions it had available to use and a description of what they did.

It responded by letting us know which tools it thought would be useful to answer the question. (In this case, it only had one choice, but it let us know that it needed it.)

Now we have to call those functions. There are two small tricky things: it gave us the name of the function we need to call, so we need to get the function from its name, and it gives the parameters that we want to call as a string.

For clarity, let's step through the workflow with just the first tool call.

### Instructions

Call the function and store the name and response.

- Get the first tool call. Assign to `tool_call`.
- Get the name of the function to call. Assign to `function_name`.
- Get the function to call from the name. Assign to `function_to_call`.
- Call it. Assign to `function_response`.

<details>
  <summary>Code hints</summary> 
  <p>
      
The name of the function to call is in the `.function.name` element of the tool.

---
      
`globals()` returns all the variables available  in a list. That allows you to get the value of a variable from its name.
      
```py
variable = globals()[name]
```
    
---
  
You can convert a JSON string into a Python variable into the actual variable with `json.loads()`.

```py
variable = json.loads(string)
```
  
---

If you have a dictionary containing all the arguments to a function, you can pass it to the function, prefixing it with `**`.
      
```py
function_to_call(**dictionary)
```      
      
  </p>
</details>

In [16]:
# Get the first tool call. Assign to tool_call.
tool_call = rsp_msg1.tool_calls[0]

# Get the name of the function to call. Assign to function_name.
function_name = tool_call.function.name

# Get the function to call from the name. Assign to function_to_call.
function_to_call = globals()[function_name]

# Get the function arguments
args = json.loads(tool_call.function.arguments)

# Call it. Assign to function_response.
function_response = function_to_call(**args)

### Instructions

Collect the function call details.



<details>
  <summary>Code hints</summary> 
  <p>
      
The function details required by GPT are in the form of a dictionary with four elements.
      
- `tool_call_id` is the `id` element of the function.
- `role` is always `"tool"`.
- `name` is the name of the function.
- `content` is the value returned by the function.
      
```py
function_call_details = {
    "tool_call_id": tool_to_call.id,
    "role": "tool",
    "name": function_name,
    "content": function_response,
}
```
      
  </p>
</details>

In [17]:
# Collect the function call details.
function_call_details = {
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": function_response
}

# See the result
function_call_details

{'tool_call_id': 'call_07MrKMsl0av1T184NytwnfuZ',
 'role': 'tool',
 'name': 'get_total_sales',
 'content': '38.0'}

We need to loop over all the function calls, collecting the details, then adding them to the chat.

Here is a convenience function to help.

### Instructions

Run this code to define the function.

In [18]:
# Run this
def get_function_call_details(tool_call):
    function_name = tool_call.function.name
    function_to_call = globals()[function_name]
    args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**args)
    return {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": function_name,
        "content": function_response
    }

### Instructions

Append the function call details to the chat.

- Loop over all the tool calls in `rsp_msg1`.
- Get the function call details using `get_function_call_details`.
- Append the function call details to the chat.

In [19]:
# Loop over all the tool calls in rsp_msg1
for tool_call in rsp_msg1.tool_calls:
    # Get the function call details
    function_call_details = get_function_call_details(tool_call)
    # Append the function call details to the chat
    chat.append(function_call_details)

# See the state of the chat
chat

[{'role': 'user',
  'content': 'Did I sell more almond croissants at my bakery in August or September?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_07MrKMsl0av1T184NytwnfuZ', function=Function(arguments='{"product": "almond croissant", "month_name": "August"}', name='get_total_sales'), type='function'), ChatCompletionMessageToolCall(id='call_WoymBthXHQLjzc3s3Ay4Uzc2', function=Function(arguments='{"product": "almond croissant", "month_name": "September"}', name='get_total_sales'), type='function')]),
 {'tool_call_id': 'call_07MrKMsl0av1T184NytwnfuZ',
  'role': 'tool',
  'name': 'get_total_sales',
  'content': '38.0'},
 {'tool_call_id': 'call_WoymBthXHQLjzc3s3Ay4Uzc2',
  'role': 'tool',
  'name': 'get_total_sales',
  'content': '25.0'}]

## Task 5: Call GPT Again, Including The Function Call Details

Now we have a conversation that includes the original request, a response from GPT saying which functions to call, and the details of the function that we called.

The final step is to call GPT again with the updated conversation.

### Instructions

Call GPT again with the updated chat. Assign to `rsp2`.

In [20]:
# Call GPT again with the updated chat. Assign to rsp2.
rsp2 = client.chat.completions.create(
    model="gpt-4o",
    messages=chat
)

# See the response
rsp2

ChatCompletion(id='chatcmpl-9jThEubxYwRKAHcLyXACUvo8BluRj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='You sold more almond croissants in August (38) than in September (25).', role='assistant', function_call=None, tool_calls=None))], created=1720625940, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_4008e3b719', usage=CompletionUsage(completion_tokens=17, prompt_tokens=104, total_tokens=121))

In [21]:
# Displatarget the updated response text 
displatarget(Markdown(rsp2.choices[0].message.content))

You sold more almond croissants in August (38) than in September (25).